In [1]:
%pylab inline
%matplotlib inline

import sys
import os
import time
import codecs
import pandas as pd
from pprint import pprint
import matplotlib.pyplot as plt
import numpy as np
import pickle


sys.path.append("/Users/Bya/git/predictEPL/utils/")
sys.path.append("/Users/Bya/git/predictEPL/config/")

import emolex
import paths
import tokenizer
import useful_methods as my_methods
import train_datas

Populating the interactive namespace from numpy and matplotlib


In [2]:
import mahotas as mh
from mahotas.features import surf
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import *
from sklearn.cluster import MiniBatchKMeans
import glob

### Load the images
convert them to grayscale, and extract the SURF descriptors

In [4]:
# os.chdir("/Volumes/Transcend/")
os.chdir("/Users/Bya/Downloads/cats-and-dogs-img/train/")
all_instance_filenames = []
all_instance_targets = []

cat_num = 0
dog_num = 0
target = 1

for f in glob.glob('*.jpg'):
    if cat_num == 1000 and dog_num == 1000:
        break

    if 'cat' in f:
        target = 1
        cat_num += 1
        if cat_num > 1000:
            cat_num = 1000
            continue
    else:
        target = 0
        dog_num += 1
        if dog_num > 1000:
            dog_num = 1000
            continue

    all_instance_filenames.append(f)
    all_instance_targets.append(target)

surf_features = []
counter = 0
for f in all_instance_filenames:
    print('Reading image:', f)
    image = mh.imread(f, as_grey=True)
    surf_features.append(surf.surf(image)[:, 5:])

train_len = int(len(all_instance_filenames) * .60)
X_train_surf_features = np.concatenate(surf_features[:train_len])
X_test_surf_feautres = np.concatenate(surf_features[train_len:])
y_train = all_instance_targets[:train_len]
y_test = all_instance_targets[train_len:]

Reading image: cat.0.jpg
Reading image: cat.1.jpg
Reading image: cat.10.jpg
Reading image: cat.100.jpg
Reading image: cat.1000.jpg
Reading image: cat.10000.jpg
Reading image: cat.10001.jpg
Reading image: cat.10002.jpg
Reading image: cat.10003.jpg
Reading image: cat.10004.jpg
Reading image: cat.10005.jpg
Reading image: cat.10006.jpg
Reading image: cat.10007.jpg
Reading image: cat.10008.jpg
Reading image: cat.10009.jpg
Reading image: cat.1001.jpg
Reading image: cat.10010.jpg
Reading image: cat.10011.jpg
Reading image: cat.10012.jpg
Reading image: cat.10013.jpg
Reading image: cat.10014.jpg
Reading image: cat.10015.jpg
Reading image: cat.10016.jpg
Reading image: cat.10017.jpg
Reading image: cat.10018.jpg
Reading image: cat.10019.jpg
Reading image: cat.1002.jpg
Reading image: cat.10020.jpg
Reading image: cat.10021.jpg
Reading image: cat.10022.jpg
Reading image: cat.10023.jpg
Reading image: cat.10024.jpg
Reading image: cat.10025.jpg
Reading image: cat.10026.jpg
Reading image: cat.10027.jpg
R

### Clustering

In [5]:
# K = 300
n_clusters = 300
print('Clustering', len(X_train_surf_features), 'features')

estimator = MiniBatchKMeans(n_clusters=n_clusters)

estimator.fit_transform(X_train_surf_features)

Clustering 769597 features


array([[ 1.25167439,  1.50775748,  1.26663778, ...,  3.67959161,
         3.55827042,  2.46968043],
       [ 1.24350896,  1.41597227,  1.1833585 , ...,  3.6454808 ,
         3.52021959,  2.46720011],
       [ 1.94381679,  1.04021683,  0.87346835, ...,  2.98090874,
         2.85786379,  3.16524385],
       ..., 
       [ 0.81412931,  1.73624083,  1.44659294, ...,  4.14606508,
         4.02218389,  2.0373949 ],
       [ 0.4554628 ,  2.43223999,  2.13359785, ...,  4.75645957,
         4.62899836,  1.35619557],
       [ 3.63880677,  1.37511466,  1.58236203, ...,  1.43528221,
         1.33029518,  4.90853827]])

### Construct vectors

300-dimensional feature vector for each instance: